In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)

# this should import glob, os, and many other standard libs
from src.utils.Notebook_imports import *

# local imports
from src.utils.Utils_io import Console_and_file_logger, init_config
from src.visualization.Visualize import show_2D_or_3D

# import external libs
import cv2
import pandas as pd

# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2

EXPERIMENT = 'temp/windowmotion'
timestemp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) # ad a timestep to each project to make repeated experiments unique

EXPERIMENTS_ROOT = 'exp/'
EXP_PATH = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT, timestemp)
MODEL_PATH = os.path.join(EXP_PATH, 'model', )
TENSORBOARD_PATH = os.path.join(EXP_PATH, 'tensorboard_logs')
CONFIG_PATH = os.path.join(EXP_PATH,'config')
HISTORY_PATH = os.path.join(EXP_PATH, 'history')
ensure_dir(MODEL_PATH)
ensure_dir(TENSORBOARD_PATH)
ensure_dir(CONFIG_PATH)
ensure_dir(HISTORY_PATH)

# define the input data paths and fold 
# first to the 4D Nrrd files, 
# second to a dataframe with a mapping of the Fold-number
# Finally the path to the metadata
DATA_PATH_SAX = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/sax/'
DF_FOLDS = '/mnt/ssd/data/gcn/metadata/df_kfold.csv'
DF_META = '/mnt/ssd/data/gcn/metadata/SAx_3D_dicomTags_phase'
FOLD = 0

# ------------------------------------------generator hyperparameters
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 2 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
DIM = [64, 128, 128] # network input params for spacing of 3, (z,y,x)
SPACING = [2, 2, 2] # if resample, resample to this spacing, (z,y,x)
T_SPACING = 55
DEPTH = 3 # depth of the encoder
FILTERS = 16 # initial number of filters, will be doubled after each downsampling block
M_POOL = [2, 2, 2]# size of max-pooling used for downsampling and upsampling
F_SIZE = [3, 3, 3] # conv filter size
# maybe we can drop this channel?
#IMG_CHANNELS = 1 # Currently our model needs that image channel
MASK_VALUES = [1, 2, 3]  #channel order: Background, RV, MYO, LV
MASK_CLASSES = len(MASK_VALUES) # no of labels
MASKING_IMAGE = False
MASKING_VALUES = [1,2,3]
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 0.8
SHUFFLE = True
RESAMPLE = True
AUGMENT_TEMP = False
AUGMENT_TEMP_RANGE = (-2,2)
RESAMPLE_T = True
HIST_MATCHING = False
SCALER = 'Standard' # MinMax Standard or Robust

Console_and_file_logger(EXPERIMENT, logging.INFO)
config = init_config(config=locals(), save=True)
print(config)

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models
['/gpu:0', '/gpu:1']


2021-05-21 11:58:31,501 INFO -------------------- Start --------------------
2021-05-21 11:58:31,502 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-05-21 11:58:31,502 INFO Log file: ./logs/temp/windowmotion.log
2021-05-21 11:58:31,502 INFO Log level for console: INFO


{'__K': None, 'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'SEED': 42, 'EXPERIMENT': 'temp/windowmotion', 'EXPERIMENTS_ROOT': 'exp/', 'EXP_PATH': 'exp/temp/windowmotion/2021-05-21_11_58', 'MODEL_PATH': 'exp/temp/windowmotion/2021-05-21_11_58/model', 'TENSORBOARD_PATH': 'exp/temp/windowmotion/2021-05-21_11_58/tensorboard_logs', 'CONFIG_PATH': 'exp/temp/windowmotion/2021-05-21_11_58/config', 'HISTORY_PATH': 'exp/temp/windowmotion/2021-05-21_11_58/history', 'DATA_PATH_SAX': '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/sax/', 'DF_FOLDS': '/mnt/ssd/data/gcn/metadata/df_kfold.csv', 'DF_META': '/mnt/ssd/data/gcn/metadata/SAx_3D_dicomTags_phase', 'FOLD': 0, 'BATCHSIZE': 2, 'GENERATOR_WORKER': 2, 'DIM': [64, 128, 128], 'SPACING': [2, 2, 2], 'T_SPACING': 55, 'DEPTH': 3, 'FILTERS': 16, 'M_POOL': [2, 2, 2], 'F_SIZE': [3, 3, 3], 'MASK_VALUES': [1, 2, 3], 'MASK_CLASSES': 3, 'MASKING_IMAGE': False, 'MASKING_VALUES': [1, 2, 3], 'BORDER_MODE': 4, 'IMG_INTERPOLATION': 1, 'MSK_INTERPOLATION': 0, 'AUGM

In [2]:
# Load SAX volumes
from src.data.Dataset import get_trainings_files
x_train_sax, y_train_sax, x_val_sax, y_val_sax =  get_trainings_files(data_path=DATA_PATH_SAX,path_to_folds_df=DF_FOLDS, fold=FOLD)
logging.info('SAX train CMR: {}, SAX train masks: {}'.format(len(x_train_sax), len(y_train_sax)))
logging.info('SAX val CMR: {}, SAX val masks: {}'.format(len(x_val_sax), len(y_val_sax)))

2021-05-21 11:58:31,584 INFO no files found, try to load with clean.nrrd/mask.nrrd pattern
2021-05-21 11:58:31,585 INFO searched in: /mnt/ssd/data/gcn/02_imported_4D_unfiltered/sax/
2021-05-21 11:58:31,593 INFO Found 278 images/masks in /mnt/ssd/data/gcn/02_imported_4D_unfiltered/sax/
2021-05-21 11:58:31,593 INFO Patients train: 209
2021-05-21 11:58:31,604 INFO Selected 209 of 278 files with 209 of 279 patients for training fold 0
2021-05-21 11:58:31,604 INFO SAX train CMR: 209, SAX train masks: 209
2021-05-21 11:58:31,605 INFO SAX val CMR: 69, SAX val masks: 69


In [3]:
# initialise a generator with these files
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import PhaseWindowGenerator
config['AUGMENT'] = True
config['RESAMPLE'] = True
batch_generator = PhaseWindowGenerator(x_train_sax, x_train_sax, config=config)
val_config = config.copy()
val_config['AUGMENT'] = False
val_config['AUGMENT_TEMP'] = False
val_config['HIST_MATCHING'] = False
validation_generator = PhaseWindowGenerator(x_val_sax, x_val_sax , config=val_config)

2021-05-21 11:58:31,760 INFO Create DataGenerator
2021-05-21 11:58:31,761 INFO Datagenerator created with: 
 shape: [64, 128, 128]
 spacing: [2, 2, 2]
 batchsize: 2
 Scaler: Standard
 Images: 209 
 Augment: True 
 Thread workers: 2
2021-05-21 11:58:31,761 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-05-21 11:58:31,775 INFO params of generator:
2021-05-21 11:58:31,776 INFO [('MASKING_IMAGE', False), ('SINGLE_OUTPUT', False), ('MASKING_VALUES', [1, 2, 3]), ('DEBUG_MODE', False), ('SCALER', 'Standard'), ('AUGMENT', True), ('SHUFFLE', True), ('RESAMPLE', True), ('SPACING', [2, 2, 2]), ('SEED', 42), ('DIM', [64, 128, 128]), ('BATCHSIZE', 2), ('MASK_VALUES', [1, 2, 3]), ('N_CLASSES', 3), ('MAX_WORKERS', 2), ('T_SPACING', 55), ('PHASES', 5), ('HIST_MATCHING', False), ('IMG_INTERPOLATION', 1), ('MSK_INTERPOLATION', 0), ('AUGMENT_TEMP', False), ('RESAMPLE_T', True), ('ISACDC', False), ('METADATA_FILE', '/mnt/ssd/data/gcn/metadata/SAx_3D_dicomTags_phase')]
2021-05

In [4]:
# visualise one batch
@interact_manual
def select_image_in_batch(batch = (0,len(batch_generator), 1), im = (0,BATCHSIZE- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    import time
    global inputs, outputs
    t1 = time.time()
    inputs, outputs = batch_generator.__getitem__(batch)
    if type(inputs)==list:inputs, outputs = inputs[0], outputs[0]
    logging.info('Preprocessing took: {:0.3f}'.format(time.time() - t1))
    print(inputs.shape, outputs.shape)
    print('selected batch : ' + str(batch))
    
    selected_input = inputs[im]
    selected_output = outputs[im]
    
    list(map(lambda x: show_2D_or_3D(img=x[::slice_n], interpol='bilinear',dpi=100,f_size=(5,5)), selected_input))
    plt.show()
    info('-'*100)
    list(map(lambda y: show_2D_or_3D(img=y[::slice_n], interpol='bilinear',dpi=100,f_size=(5,5)), selected_output))
    plt.show()

    plt.hist(selected_input.flatten(), bins=np.linspace(0.1,1,20))
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))
    



interactive(children=(IntSlider(value=52, description='batch', max=104), IntSlider(value=0, description='im', …

In [5]:
# visualise one batch
@interact_manual
def select_image_in_batch(batch = (0,len(batch_generator), 1), im = (0,BATCHSIZE- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    import time
    global inputs, outputs
    t1 = time.time()
    inputs, outputs = batch_generator.__getitem__(batch)
    if type(inputs)==list:inputs, outputs = inputs[0], outputs[0]
    logging.info('Preprocessing took: {:0.3f}'.format(time.time() - t1))
    print(inputs.shape, outputs.shape)
    print('selected batch : ' + str(batch))
    
    selected_input = inputs[im]
    selected_output = outputs[im]
    
    # plot the pre (t-1) and post (t+1) volume of each cardiac phase and the difference
    _ = list(map(lambda x : show_two_timesteps(x),  zip(['ED#', 'MS#', 'ES#', 'PF#', 'MD#'],np.stack([selected_input, selected_output], axis=1))))

    plt.hist(selected_input.flatten(), bins=np.linspace(0.1,1,20))
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))
        
def show_two_timesteps(x):
    # x is a tuple of phase (string), ndarray with the following shape: 2,z,x,y --> first and second timestep of the motion window
    p, x = x
    inter, dpi, f_size = 'bilinear', 100, (5,5)
    logging.info('Phase: {}'.format(p))
    show_2D_or_3D(img=x[0], interpol=inter,dpi=dpi,f_size=f_size);plt.show()
    show_2D_or_3D(img=x[1], interpol=inter,dpi=dpi,f_size=f_size);plt.show()
    show_2D_or_3D(img=x[0] - x[1], interpol=None,dpi=dpi,f_size=f_size);plt.show()
    logging.info("-"*20)
    


interactive(children=(IntSlider(value=52, description='batch', max=104), IntSlider(value=0, description='im', …

In [6]:
from src.models.Models import create_RegistrationModel

import tensorflow

if tensorflow.distribute.has_strategy():
    strategy = tensorflow.distribute.get_strategy()
else:
    # distribute the training with the "mirrored data"-paradigm across multiple gpus if available, if not use gpu 0
    strategy = tensorflow.distribute.MirroredStrategy(devices=config.get('GPUS', ["/gpu:0"]))
with strategy.scope():
    model = create_RegistrationModel(config=config)

using tensorflow, need to monkey patch
tf.python.backend.slice overwritten by monkey patch
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2021-05-21 11:58:32,728 INFO Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


(None, 64, 128, 128, 1)
(None, 64, 128, 128, 1)
Instructions for updating:
Use fn_output_signature instead


2021-05-21 11:58:33,853 WARNING From /home/sven/anaconda3/envs/dcmr/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:574: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead


In [7]:
model.summary()

Model: "simpleregister"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 64, 128,  0                                            
__________________________________________________________________________________________________
tf_op_layer_unstack (TensorFlow [(None, 64, 128, 128 0           input_1[0][0]                    
__________________________________________________________________________________________________
unet (Functional)               (None, 64, 128, 128, 1604832     tf_op_layer_unstack[0][3]        
                                                                 tf_op_layer_unstack[0][1]        
                                                                 tf_op_layer_unstack[0][2]        
                                                                 tf_op_layer_unstack[

In [8]:
import tensorflow
from tensorflow.keras.losses import mse
from src.utils.Metrics import Grad, MSE_
if 'strategy' in locals():
    pass
else:
    # distribute the training with the "mirrored data"-paradigm across multiple gpus if available, if not use gpu 0
    strategy = tensorflow.distribute.MirroredStrategy(devices=config.get('GPUS', ["/gpu:0"]))
with strategy.scope():

    losses = [mse, Grad('l1').loss]
    weights = [1,0.01]
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(), loss=losses, loss_weights=weights)

In [9]:
from src.utils.KerasCallbacks import get_callbacks

results = model.fit(
    x=batch_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks = get_callbacks(config, batch_generator,validation_generator),
    initial_epoch=0,

    verbose=1)

Epoch 1/100
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


2021-05-21 11:58:36,569 WARNING From /home/sven/anaconda3/envs/dcmr/lib/python3.8/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


INFO:tensorflow:batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1


2021-05-21 11:58:43,746 INFO batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

2021-05-21 11:58:53,232 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2021-05-21 11:58:53,234 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


104/104 [==============================] - ETA: 0s - loss: 0.1343 - tf_op_layer_stack_loss: 0.1324 - tf_op_layer_stack_1_loss: 0.1890
Epoch 00001: loss improved from inf to 0.13433, saving model to exp/temp/windowmotion/2021-05-21_11_58/model/model.h5
104/104 [==============================] - 224s 2s/step - loss: 0.1343 - tf_op_layer_stack_loss: 0.1324 - tf_op_layer_stack_1_loss: 0.1890 - val_loss: 0.1342 - val_tf_op_layer_stack_loss: 0.1341 - val_tf_op_layer_stack_1_loss: 0.0120 - lr: 0.0010
Epoch 2/100
104/104 [==============================] - ETA: 0s - loss: 0.1094 - tf_op_layer_stack_loss: 0.1090 - tf_op_layer_stack_1_loss: 0.0344
Epoch 00002: loss improved from 0.13433 to 0.10937, saving model to exp/temp/windowmotion/2021-05-21_11_58/model/model.h5
104/104 [==============================] - 233s 2s/step - loss: 0.1094 - tf_op_layer_stack_loss: 0.1090 - tf_op_layer_stack_1_loss: 0.0344 - val_loss: 0.1277 - val_tf_op_layer_stack_loss: 0.1275 - val_tf_op_layer_stack_1_loss: 0.0223

KeyboardInterrupt: 

In [10]:
inputs.shape

(2, 5, 64, 128, 128, 1)

In [11]:
pred = model.predict(x=inputs)

In [12]:
transformed, flow = pred
print(transformed.shape)
print(flow.shape)

(2, 5, 64, 128, 128, 1)
(2, 5, 64, 128, 128, 3)


# save as numpy

In [26]:
np.save('reports/flow.npz',flow)
np.save('reports/input.npz',inputs)
np.save('reports/target.npz',outputs)

# Save as nifti

- 5 x 3D input/output/flow
- z x 2D+t input/output/flow

In [15]:
def save_3d(nda, fname):
    # save one flowfield
    sitk_img = sitk.GetImageFromArray(nda, isVector=False)
    sitk.WriteImage(sitk_img, fname)

def save_all_3d_vols(inputs, outputs, flow, export_path='reports/exampleflows', exp='example_flows'):
    experiment_ = '{}/{}'.format(EXP_PATH, exp)
    info(experiment_)
    ensure_dir(experiment_)
    flowname = os.path.join(experiment_, '_flow.nii')
    firstfilename = os.path.join(experiment_, '_cmr.nii')
    secondfilename = os.path.join(experiment_, '_targetcmr.nii')
    
    # invert the axis
    flow = np.einsum('tzyxc->cxyzt', flow)
    inputs = np.einsum('tzyxc->cxyzt', inputs)
    outputs = np.einsum('tzyxc->cxyzt', outputs)
    
    _ = [save_3d(flow[...,t], flowname.replace('.nii', '_{}_.nii'.format(t))) for t in range(flow.shape[-1])]
    _ = [save_3d(inputs[...,t], firstfilename.replace('.nii', '_{}_.nii'.format(t))) for t in range(inputs.shape[-1])]
    _ = [save_3d(outputs[...,t], secondfilename.replace('.nii', '_{}_.nii'.format(t))) for t in range(outputs.shape[-1])]
    
    _ = [save_3d(flow[...,t,:], flowname.replace('.nii', '_sequence_{}_.nii'.format(t))) for t in range(flow.shape[-2])]
    _ = [save_3d(inputs[...,t,:], firstfilename.replace('.nii', '_sequence_{}_.nii'.format(t))) for t in range(inputs.shape[-2])]
    _ = [save_3d(outputs[...,t,:], secondfilename.replace('.nii', '_sequence_{}_.nii'.format(t))) for t in range(outputs.shape[-2])]
    
save_all_3d_vols(inputs[0], outputs[0], flow[0])

2021-05-21 14:28:14,645 INFO exp/temp/windowmotion/2021-05-21_11_58/example_flows


# save one example 3D flowfield as nifti

In [13]:
experiment_ = '{}/{}'.format(EXP_PATH, 'example_flows')
ensure_dir(experiment_)
flowname = os.path.join(experiment_, '_flow.nii')
firstfilename = os.path.join(experiment_, '_cmr.nii')
secondfilename = os.path.join(experiment_, '_targetcmr.nii')
t = 3

In [16]:
flow_single = flow[0]
print(flow_single.shape)
flow_single = np.einsum('tzyxc->cxyzt', flow_single)
flow_single = flow_single[...,t]
flow_sitk = sitk.GetImageFromArray(flow_single, isVector=False)
print(flow_sitk.GetSize())
sitk.WriteImage(flow_sitk, flowname)

(5, 32, 128, 128, 3)
(32, 128, 128, 3)


In [17]:
# save the first time step
inputs_single = inputs[0]
print(inputs_single.shape)
inputs_single = np.einsum('tzyxc->cxyzt', inputs_single)
inputs_single = inputs_single[...,t]
input_sitk = sitk.GetImageFromArray(inputs_single, isVector=False)
print(input_sitk.GetSize())
sitk.WriteImage(input_sitk, firstfilename)

(5, 32, 128, 128, 1)
(32, 128, 128, 1)


In [18]:
# save the second time step
outputs_single = outputs[0]
print(outputs_single.shape)
outputs_single = np.einsum('tzyxc->cxyzt', outputs_single)
outputs_single = outputs_single[...,t]
output_sitk = sitk.GetImageFromArray(outputs_single, isVector=False)
print(output_sitk.GetSize())
sitk.WriteImage(output_sitk, secondfilename)

(5, 32, 128, 128, 1)
(32, 128, 128, 1)


# save one example 2D+T flowfield as nifti

In [31]:
temporalflowname = flowname.replace('flow.nii', 'tempflow.nii')
temporalfirstfilename = firstfilename.replace('cmr.nii', 'tempcmr.nii')
temporalsecondfilename = secondfilename.replace('targetcmr.nii', 'temptargetcmr.nii')
slice_ = 15

In [32]:
# save one 2D+t flowfield
flow_single = flow[0]
print(flow_single.shape)
flow_single = np.einsum('tzyxc->cxyzt', flow_single)
print(flow_single.shape)
flow_single = flow_single[:,:,:,slice_,:]
print(flow_single.shape)
flow_sitk = sitk.GetImageFromArray(flow_single, isVector=False)
print(flow_sitk.GetSize())
print(flow_sitk.GetNumberOfComponentsPerPixel())
sitk.WriteImage(flow_sitk, temporalflowname)

(5, 32, 128, 128, 3)
(3, 128, 128, 32, 5)
(3, 128, 128, 5)
(5, 128, 128, 3)
1


# save 4D flow - not working so far

In [130]:
# save the second time step
temp = flow[0]
print(temp.shape)
#temp = np.einsum('tzyxc->ctzyx', temp)
#temp = [temp[:,:,:,i,:] for i in range(outputs_single.shape[-2])]
temp = sitk.JoinSeries([sitk.GetImageFromArray(img, isVector=False) for img in temp])
print(temp.GetSize())
print(temp.GetNumberOfComponentsPerPixel())
sitk.WriteImage(temp, os.path.join(experiment_, '4dflow.nii'))

(5, 30, 128, 128, 3)
(3, 128, 128, 30, 5)
1


In [138]:
# stack two slices - tests
flow_single = flow[0]
print(flow_single.shape)
flow_single = flow_single[:,slice_,:,:,:]
print(flow_single.shape)
flow_single = np.einsum('tyxc->cxyt', flow_single)
print(flow_single.shape)
flow_single = np.concatenate([flow_single,flow_single], axis=-1)
print(flow_single.shape)
flow_sitk = sitk.GetImageFromArray(flow_single, isVector=True)
print(flow_sitk.GetSize())
print(flow_sitk.GetNumberOfComponentsPerPixel())
sitk.WriteImage(flow_sitk, temporalflowname)

(5, 30, 128, 128, 3)
(5, 128, 128, 3)
(3, 128, 128, 5)
(3, 128, 128, 10)
(128, 128, 3)
10


In [151]:
# stack z x 2D+T flows - works
flow_single = flow[0]
print(flow_single.shape)
flows = np.split(flow_single,indices_or_sections=flow_single.shape[1], axis=1)
print(flows[0].shape)
flows = [np.einsum('tyxc->cxyt', np.squeeze(f)) for f in flows]
print(flows[0].shape)
flows = np.concatenate(flows, axis=-1)
print(flows.shape)
flow_sitk = sitk.GetImageFromArray(flows, isVector=False)
print(flow_sitk.GetSize())
print(flow_sitk.GetNumberOfComponentsPerPixel())
sitk.WriteImage(flow_sitk, experiment_+'/stacked_2d+t.nii')

(5, 30, 128, 128, 3)
(5, 1, 128, 128, 3)
(3, 128, 128, 5)
(3, 128, 128, 150)
(150, 128, 128, 3)
1


In [162]:
# stack z x 2D+T slices - tests
flow_single = np.squeeze(inputs[0])
print(flow_single.shape)
flow_single = np.reshape(flow_single,(150,128,128))
flow_sitk = sitk.GetImageFromArray(flow_single, isVector=False)
print(flow_sitk.GetSize())
print(flow_sitk.GetNumberOfComponentsPerPixel())
sitk.WriteImage(flow_sitk, experiment_+'/stacked_1st_cmr.nii')

(5, 30, 128, 128)
(128, 128, 150)
1


In [78]:
# save one 2D+t flowfield
flow_single = flow[0]
flow_single = np.einsum('tzyxc->cxyzt', flow_single)
flows = np.split(flow_single, indices_or_sections=5, axis=-1)
flows = [sitk.DisplacementFieldTransform(f) for f in flows]
flow_sitk = sitk.JoinSeries(flows)
print(flow_sitk.GetSize())
sitk.WriteTransform(flow_sitk, 'reports/examplefullflow_4d.nrrd')

TypeError: Wrong number or type of arguments for overloaded function 'new_DisplacementFieldTransform'.
  Possible C/C++ prototypes are:
    itk::simple::DisplacementFieldTransform::DisplacementFieldTransform(unsigned int)
    itk::simple::DisplacementFieldTransform::DisplacementFieldTransform(itk::simple::Image &)
    itk::simple::DisplacementFieldTransform::DisplacementFieldTransform(itk::simple::DisplacementFieldTransform const &)
    itk::simple::DisplacementFieldTransform::DisplacementFieldTransform(itk::simple::Transform const &)


In [164]:
# save one flowfield
flow_single = flow[0]
print(flow_single.shape)
#flow_single = np.einsum('tzyxc->ctzyx', flow_single)
print(flow_single.shape)
#flow_single = flow_single[...,t]

#flow_single = np.reshape(flow_single, newshape=(15,30,128,128), order='F')
print(flow_single.shape)
flow_sitk = sitk.GetImageFromArray(flow_single, isVector=False)
print(flow_sitk.GetSize())
sitk.WriteImage(flow_sitk, flowname)

(5, 30, 128, 128, 3)
(5, 30, 128, 128, 3)
(5, 30, 128, 128, 3)
(3, 128, 128, 30, 5)


In [ ]:
flow_sitk = sitk.GetImageFromArray(flow[0], isVector=False)
sitk.WriteImage(flow_sitk, 'reports/example2flow.ni.gz')

"""inputs_sitk = sitk.GetImageFromArray(inputs)
outputs_sitk = sitk.GetImageFromArray(outputs)"""

In [16]:
from src.visualization.Visualize import create_quiver_plot

@interact
def show_pred(t=(0,5), slice_z=(0,5), debug_=False):
    info('input')
    if debug_:list(map(lambda x: show_2D_or_3D(img=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)), inputs[0][t:t+1]));plt.show()
    info('target')
    if debug_:list(map(lambda x: show_2D_or_3D(img=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)), outputs[0][t:t+1]));plt.show()
    diff = inputs - outputs
    info('diff input')
    if debug_:list(map(lambda x: show_2D_or_3D(img=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)), diff[0][t:t+1]));plt.show()
    info('pred')
    if debug_:list(map(lambda x: show_2D_or_3D(img=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)), transformed[0][t:t+1]));plt.show()
    info('diff pred')
    diff = transformed - outputs
    if debug_:list(map(lambda x: show_2D_or_3D(img=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)), diff[0][t:t+1]));plt.show()
    if debug_:fig,axes = plt.subplots(1,flow.shape[2]//slice_z, figsize=(24,2))
    if debug_:list(map(lambda x: create_quiver_plot(flowfield_2d=x[1],indexing='ij', scale=0.5, N=1,ax=x[0]), zip(axes,flow[0][t][::slice_z]))); plt.show() # third timestep
    list(map(lambda x: show_2D_or_3D(mask=x[::slice_z], interpol='bilinear',dpi=100,f_size=(5,5)) and plt.show(), flow[0][t:t+1]))
    plt.show()
    if debug_:plt.hist(flow[0].flatten())

interactive(children=(IntSlider(value=2, description='t', max=5), IntSlider(value=2, description='slice_z', ma…